# Pass

In [ ]:
import tvm
from tvm import relay
import numpy as np
from tvm.contrib import graph_executor

In [ ]:
# 建構神經網路
## 建構BN
# 利用relay提供的

def batch_norm(data,gamma=None,beta=None,moving_mean=None,moving_var=None,**kwargs):
    name = kwargs.get("name")
    kwargs.pop("name")
    if not gamma:
        gamma = relay.var(name + "_gamma")
    if not beta:
        beta = relay.var(name + "_beta")
    if not moving_mean:
        moving_mean = relay.var(name + "_moving_mean")
    if not moving_var:
         moving_var = relay.var(name + "_moving_var")
    
    return relay.nn.batch_norm(data,
                               gamma=gamma,
                               beta=beta,
                               moving_mean=moving_mean,
                               moving_var=moving_var,
                               **kwargs)[0]


# 建構卷積
def conv2d(data, weight=None, **kwargs):
    name = kwargs.get("name")
    kwargs.pop("name")
    if not weight:
        weight = relay.var(name + "_weight")
    return relay.nn.conv2d(data, weight, **kwargs)

#建構卷積+BN+ReLU的simpleNet
def simplenet(data, name, channels, kernel_size=(3, 3), strides=(1, 1),
               padding=(1, 1), epsilon=1e-5):
    # 卷積
    conv = conv2d(
        data=data,
        channels=channels,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        data_layout='NCHW',
        name=name+'_conv') 
    
    # BN
    bn = batch_norm(data=conv, epsilon=epsilon, name=name + '_bn')
    # relu
    act = relay.nn.relu(data=bn)
    return act

# data shape
data_shape = (1, 3, 224, 224)

# 卷積的kernel_size
kernel_shape = (32, 3, 3, 3)

# input的placeholder
dtype = "float32"
data = relay.var("data", shape=data_shape, dtype=dtype)


act = simplenet(data, "graph", 32, strides=(2, 2))
func = relay.Function(relay.analysis.free_vars(act), act)

print(func)


In [ ]:
# 隨機產生資料
np_data = np.random.uniform(-1, 1, (1, 3, 224, 224))

# 產生模型參數值
params = {
    "graph_conv_weight": tvm.nd.array(np.random.uniform(-1, 1, (32, 3, 3, 3)).astype(dtype)),
    "graph_bn_gamma": tvm.nd.array(np.random.uniform(-1, 1, (32)).astype(dtype)),
    "graph_bn_beta": tvm.nd.array(np.random.uniform(-1, 1, (32)).astype(dtype)),
    "graph_bn_moving_mean": tvm.nd.array(np.random.uniform(-1, 1, (32)).astype(dtype)),
    "graph_bn_moving_var": tvm.nd.array(np.random.uniform(-1, 1, (32)).astype(dtype)),
}

# 執行Pass圖優化
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(func, "llvm", params=params)



In [ ]:
#coding=utf-8

# 運行計算圖
dev = tvm.cpu(0)
dtype = "float32"
m = graph_executor.GraphModule(lib["default"](dev))
# set inputs
m.set_input("data", tvm.nd.array(np_data.astype(dtype)))
# execute
m.run()
# get outputs
tvm_output = m.get_output(0)

# 圖優化

## 手動圖優化

In [1]:
import numpy as np
import tvm
from tvm import te
import tvm.relay as relay

In [2]:
def example():
    shape = (1, 64, 54, 54)
    c_data = np.empty(shape).astype("float32")
    c = relay.const(c_data)
    
    weight = relay.var("weight", shape=(64, 64, 3, 3))
    x = relay.var("x", relay.TensorType((1, 64, 56, 56), "float32"))
    
    conv = relay.nn.conv2d(x, weight)
    y = relay.add(c, c)
    y = relay.multiply(y, relay.const(2, "float32"))
    y = relay.add(conv, y)
    z = relay.add(y, c)
    z1 = relay.add(y, c)
    z2 = relay.add(z, z1)
    return relay.Function([x, weight], z2)

In [3]:
# functions for optimization.
f = example()
mod = tvm.IRModule.from_expr(f)

print(type(mod))
#<class 'tvm.ir.module.IRModule'>

<class 'tvm.ir.module.IRModule'>


In [4]:
# 尚未優化的結果
print(mod.show())

/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


### Pass

#### Constant Fold

In [253]:
# 應用Constant Fold
# Now we can apply constant folding on the module.
# fold_const here is a callback that doesn't take any parameters.
# 將Constant Fold應用在tvm.ir.module.IRModule
fold_const = relay.transform.FoldConstant()
print(fold_const)

Run Function pass: FoldConstant at the optimization level 2


In [254]:
# Constant Fold在IRModule
# Then, we can invoke the pass on the given module. 
#Note that the constant folding pass works at the function-level. 
# That being said, each function in the module will be applied with the optimization. 
# Users don't need to iterate through individual functions manually to apply this pass.
mod_f = fold_const(mod)
# We can see from the updated program that the constants are folded.
print(mod_f.show())

/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


#### Common Subexpression Elimination

In [255]:
# 消除z和z1的Common Subexpression Elimination
# 以類似方式應用更多優化，例如，消除z和z1的通用表達式
EliminateCommonSubexpr = relay.transform.EliminateCommonSubexpr()

# 執行Common Subexpression Elimination
mod_e = EliminateCommonSubexpr(mod)
print(mod_e.show())

/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


### 使用 Sequential 來應用 Passes 序列



#### 使用 Sequential 來應用 Passes 序列
tvm.transform.Sequential([pass])

In [246]:
f = example()
mod = tvm.IRModule.from_expr(f)
# Glob the interested passes.
seq = tvm.transform.Sequential([relay.transform.FoldConstant(),
                                relay.transform.EliminateCommonSubexpr(),
                                relay.transform.FuseOps(fuse_opt_level=2)])

# 執行Pass序列
mod_seq = seq(mod)
print(mod_seq)

def @main(%x: Tensor[(1, 64, 56, 56), float32] /* ty=Tensor[(1, 64, 56, 56), float32] */, %weight: Tensor[(64, 64, 3, 3), float32] /* ty=Tensor[(64, 64, 3, 3), float32] */) -> Tensor[(1, 64, 54, 54), float32] {
  %4 = fn (%p0: Tensor[(1, 64, 56, 56), float32] /* ty=Tensor[(1, 64, 56, 56), float32] */, %p1: Tensor[(64, 64, 3, 3), float32] /* ty=Tensor[(64, 64, 3, 3), float32] */, %p2: Tensor[(1, 64, 54, 54), float32] /* ty=Tensor[(1, 64, 54, 54), float32] */, %p3: Tensor[(1, 64, 54, 54), float32] /* ty=Tensor[(1, 64, 54, 54), float32] */, Primitive=1) -> Tensor[(1, 64, 54, 54), float32] {
    %0 = nn.conv2d(%p0, %p1, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
    %1 = add(%0, %p2) /* ty=Tensor[(1, 64, 54, 54), float32] */;
    %2 = add(%1, %p3) /* ty=Tensor[(1, 64, 54, 54), float32] */;
    %3 = add(%1, %p3) /* ty=Tensor[(1, 64, 54, 54), float32] */;
    add(%2, %3) /* ty=Tensor[(1, 64, 54, 54), float32] */
  } /* ty=fn (Tensor[(1, 64, 56, 56), float32], Tensor[(64

從轉換後的Relay程序中，可以看到仍然有兩個相同的加法運算。這是EliminateCommonSubexpr 未實際執行。

原因別小於或者等於2的pass才會在tvm.transform.Sequential下默認執行。

但是，Pass Infra 提供了一個配置接口，供用戶自定義他們想要執行的優化級別。

#### 自己定義要優化的級別
下面的pass提供了一個配置界面，供用戶自定義要執行的優化級別。



In [282]:
with tvm.transform.PassContext(opt_level=3):
    mod4 = seq(mod)

print(mod4.show())

/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


範例：Sequential加入自行定義要優化的級別

加入tvm.transform.Sequential()

In [48]:
f = example()
mod = tvm.IRModule.from_expr(f)
# Glob the interested passes.
seq = tvm.transform.Sequential([relay.transform.FoldConstant(),
                                relay.transform.EliminateCommonSubexpr(),
                                relay.transform.FuseOps(fuse_opt_level=2)])

# 執行Pass序列
with tvm.transform.PassContext(opt_level=3):
    mod2 = seq(mod)

print(mod2.script())

def @main(%x: Tensor[(1, 64, 56, 56), float32] /* ty=Tensor[(1, 64, 56, 56), float32] */, %weight: Tensor[(64, 64, 3, 3), float32] /* ty=Tensor[(64, 64, 3, 3), float32] */) -> Tensor[(1, 64, 54, 54), float32] {
  %3 = fn (%p0: Tensor[(1, 64, 56, 56), float32] /* ty=Tensor[(1, 64, 56, 56), float32] */, %p1: Tensor[(64, 64, 3, 3), float32] /* ty=Tensor[(64, 64, 3, 3), float32] */, %p2: Tensor[(1, 64, 54, 54), float32] /* ty=Tensor[(1, 64, 54, 54), float32] */, %p3: Tensor[(1, 64, 54, 54), float32] /* ty=Tensor[(1, 64, 54, 54), float32] */, Primitive=1) -> Tensor[(1, 64, 54, 54), float32] {
    %0 = nn.conv2d(%p0, %p1, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
    %1 = add(%0, %p2) /* ty=Tensor[(1, 64, 54, 54), float32] */;
    %2 = add(%1, %p3) /* ty=Tensor[(1, 64, 54, 54), float32] */;
    add(%2, %2) /* ty=Tensor[(1, 64, 54, 54), float32] */
  } /* ty=fn (Tensor[(1, 64, 56, 56), float32], Tensor[(64, 64, 3, 3), float32], Tensor[(1, 64, 54, 54), float32], Tensor[(

可以看到僅保留了兩個相同的加法之一。

用戶可以使用disabled_pass配置有選擇地禁用某些pass，這類似於通用編譯器（例如Clang和GCC）使用的-fno-xxx選項。例如，可以禁用EliminateCommonSubexpr，如下所示。打印的模塊將再次顯示兩個相同的加法運算。

In [275]:
with tvm.transform.PassContext(opt_level=3, disabled_pass=["EliminateCommonSubexpr"]):
    mod3 = seq(mod)

print(mod3.show())

/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


#### Memory Allocation
要先經過註冊才可以使用

In [5]:
# Memory Allocation註冊
@relay.op.register_alter_op_layout("nn.conv2d", level=101)
def alter_conv2d(attrs, inputs, tinfos, out_type):
    data, weight = inputs
    new_attrs = dict(attrs)
    new_attrs["data_layout"] = "NCHW16c"
    return relay.nn.conv2d(data, weight, **new_attrs)


#### 執行layout alteration pass優化

In [6]:
seq1 = tvm.transform.Sequential([relay.transform.AlterOpLayout()])
with tvm.transform.PassContext(opt_level=3):
    with tvm.target.Target("llvm"):
        mod5 = seq1(mod)

print(mod5.show())


/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


## 使用Python Decorator實施手動Pass

範例說明如何使用Python Decorator pass

In [285]:
@relay.transform.function_pass(opt_level=1)
class CustomPipeline:
    """Simple test function to replace one argument to another."""

    def __init__(self, multiplier):
        self.multiplier = multiplier

    # This function can define a pass.
    def transform_function(self, func, mod, ctx):
        obj = self

        class ReplaceConstant(tvm.relay.ExprMutator):
            def visit_constant(self, c):
                return relay.multiply(obj.multiplier, c)

        return ReplaceConstant().visit(func)




In [287]:
f = example()
mod = tvm.IRModule.from_expr(f)
custom_pass = CustomPipeline(multiplier=relay.const(3, "float32"))
assert custom_pass.info.name == "CustomPipeline"
mod3 = custom_pass(mod)
print(mod3.show())

/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


# Sequential pass

## 未優化前結果

In [225]:
import tvm
from tvm.relay import transform
import tvm.relay as relay
import numpy as np
from tvm.contrib import graph_runtime
 
 
def batch_norm_infer(data,
                    gamma=None,
                    beta=None,
                    moving_mean=None,
                    moving_var=None,
                    **kwargs):
    name = kwargs.get("name")
    kwargs.pop("name")
    if not gamma:
        gamma = relay.var(name + "_gamma")
    if not beta:
        beta = relay.var(name + "_beta")
    if not moving_mean:
        moving_mean = relay.var(name + "_moving_mean")
    if not moving_var:
        moving_var = relay.var(name + "_moving_var")
    return relay.nn.batch_norm(data,
                            gamma=gamma,
                            beta=beta,
                            moving_mean=moving_mean,
                            moving_var=moving_var,
                            **kwargs)[0]
 
def conv2d(data, weight=None, **kwargs):
    name = kwargs.get("name")
    kwargs.pop("name")
    if not weight:
        weight = relay.var(name + "_weight")
    return relay.nn.conv2d(data, weight, **kwargs)
 
 
def conv_block(data, name, channels, kernel_size=(3, 3), strides=(1, 1),
            padding=(1, 1), epsilon=1e-5):
    conv = conv2d(
        data=data,
        channels=channels,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        data_layout='NCHW',
        name=name+'_conv')
    bn = batch_norm_infer(data=conv, epsilon=epsilon, name=name + '_bn')
    act = relay.nn.relu(data=bn)
    return act
 
 
data_shape = (1, 3, 224, 224)
kernel_shape = (32, 3, 3, 3)
dtype = "float32"
data = relay.var("data", shape=data_shape, dtype=dtype)
act = conv_block(data, "graph", 32, strides=(2, 2))
func = relay.Function(relay.analysis.free_vars(act),act)
print(type(func))
 #<class 'tvm.relay.function.Function'>


<class 'tvm.relay.function.Function'>


In [226]:
mod = tvm.IRModule.from_expr(func)
print(mod)

def @main(%data: Tensor[(1, 3, 224, 224), float32], %graph_conv_weight, %graph_bn_gamma, %graph_bn_beta, %graph_bn_moving_mean, %graph_bn_moving_var) {
  %0 = nn.conv2d(%data, %graph_conv_weight, strides=[2, 2], padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %1 = nn.batch_norm(%0, %graph_bn_gamma, %graph_bn_beta, %graph_bn_moving_mean, %graph_bn_moving_var);
  %2 = %1.0;
  nn.relu(%2)
}



In [227]:
# 推斷 expr 的類型。
mod = relay.transform.InferType()(mod)
print(mod)

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %graph_conv_weight: Tensor[(32, 3, 3, 3), float32] /* ty=Tensor[(32, 3, 3, 3), float32] */, %graph_bn_gamma: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_beta: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_moving_mean: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_moving_var: Tensor[(32), float32] /* ty=Tensor[(32), float32] */) -> Tensor[(1, 32, 112, 112), float32] {
  %0 = nn.conv2d(%data, %graph_conv_weight, strides=[2, 2], padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 112, 112), float32] */;
  %1 = nn.batch_norm(%0, %graph_bn_gamma, %graph_bn_beta, %graph_bn_moving_mean, %graph_bn_moving_var) /* ty=(Tensor[(1, 32, 112, 112), float32], Tensor[(32), float32], Tensor[(32), float32]) */;
  %2 = %1.0 /* ty=Tensor[(1, 32, 112, 112), float32] */;
  nn.relu(%2) /* ty=Tensor[(1, 32, 112, 112), float32] */
}



函數：
tvm.relay.transform.InferType()
說明：
Infer the type of an expr.
參數：
Returns
ret – The registered type inference pass.
Return type:
tvm.transform.Pass

In [228]:
# 隨機產生模型參數值
shape_dict = {v.name_hint : v.checked_type for v in mod["main"].params}
np.random.seed(0)
params = {}
target ='llvm'
dev = tvm.device(target, 0)
#dev = tvm.cpu(0)

for k, v in shape_dict.items():
    if k == "data":
        continue
    init_value = np.random.uniform(-1, 1, v.concrete_shape).astype(v.dtype)
    params[k] = tvm.nd.array(init_value, device=dev)
    


In [229]:
# 計算圖結構和參數
print("Relay module function:n", mod.astext(show_meta_data=False))
print("TVM parameters:n", params.keys())
 

Relay module function:n #[version = "0.0.5"]
def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %graph_conv_weight: Tensor[(32, 3, 3, 3), float32] /* ty=Tensor[(32, 3, 3, 3), float32] */, %graph_bn_gamma: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_beta: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_moving_mean: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_moving_var: Tensor[(32), float32] /* ty=Tensor[(32), float32] */) -> Tensor[(1, 32, 112, 112), float32] {
  %0 = nn.conv2d(%data, %graph_conv_weight, strides=[2, 2], padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 112, 112), float32] */;
  %1 = nn.batch_norm(%0, %graph_bn_gamma, %graph_bn_beta, %graph_bn_moving_mean, %graph_bn_moving_var) /* ty=(Tensor[(1, 32, 112, 112), float32], Tensor[(32), float32], Tensor[(32), float32]) */;
  %2 = %1.0 /* ty=Tensor[(1, 32, 112, 112), float32] */;
  nn.relu(%2) /* ty=T

In [230]:
# 執行op層級設定
with relay.build_config(opt_level=3):
    graph, lib, params = relay.build(mod, target, params=params)

/var/folders/kv/q1d3k1ys06z4_0qn45hbkqrr0000gn/T/ipykernel_72441/2787215358.py:3: DeprecationWarning: legacy graph executor behavior of producing json / lib / params will be removed in the next release. Please see documents of tvm.contrib.graph_executor.GraphModule for the  new recommended usage.
  graph, lib, params = relay.build(mod, target, params=params)


In [231]:
#print("TVM graph:n", graph)
#print("TVM parameters:n", params.keys())
# print("TVM compiled target function:n", lib.get_source())

# 隨機產生input值
data_tvm = tvm.nd.array((np.random.uniform(-1, 1, size=data_shape)).astype(dtype))

# 執行計算圖
module = graph_runtime.create(graph, lib, dev)

module.set_input('data', data_tvm)
module.set_input(**params)
module.run()
output = module.get_output(0)


In [235]:
# 進行圖優化
# mod = tvm.IRModule.from_expr(func)
print(type(mod['main']))
# <class 'tvm.relay.function.Function'>
mod['main'] = my_optimize(mod['main'], params)
print(type(mod))

<class 'tvm.relay.function.Function'>
<class 'tvm.ir.module.IRModule'>


In [236]:
#mod = relay.transform.InferType()(mod)
#print(mod)

In [237]:
# 執行圖優化結果
#target = "llvm"
#ctx = tvm.context(target, 0)

target ='llvm'
dev = tvm.device(target, 0)
#print("Relay module function:n", mod.astext(show_meta_data=False))
#print("TVM parameters:n", params.keys())
 


In [238]:
# relay build設定
with relay.build_config(opt_level=3):
    graph, lib, params = relay.build(mod, target, params=params)

/var/folders/kv/q1d3k1ys06z4_0qn45hbkqrr0000gn/T/ipykernel_72441/3882782940.py:3: DeprecationWarning: legacy graph executor behavior of producing json / lib / params will be removed in the next release. Please see documents of tvm.contrib.graph_executor.GraphModule for the  new recommended usage.
  graph, lib, params = relay.build(mod, target, params=params)


In [239]:
# 查看圖結構
#print("TVM graph:n", graph)
#print("TVM parameters:n", params.keys())
# print("TVM compiled target function:n", lib.get_source())

In [240]:
# 執行運算
module = graph_runtime.create(graph, lib, dev)
data_tvm = tvm.nd.array((np.random.uniform(-1, 1, size=data_shape)).astype(dtype))
module.set_input('data', data_tvm)
module.set_input(**params)
module.run()
output = module.get_output(0)

In [242]:
print("Relay module function:n", mod.astext(show_meta_data=False))


Relay module function:n #[version = "0.0.5"]
def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %graph_conv_weight: Tensor[(32, 3, 3, 3), float32] /* ty=Tensor[(32, 3, 3, 3), float32] */, %graph_bn_gamma: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_beta: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_moving_mean: Tensor[(32), float32] /* ty=Tensor[(32), float32] */, %graph_bn_moving_var: Tensor[(32), float32] /* ty=Tensor[(32), float32] */) -> Tensor[(1, 32, 112, 112), float32] {
  %0 = add(%graph_bn_moving_var, 1e-05f /* ty=float32 */) /* ty=Tensor[(32), float32] */;
  %1 = sqrt(%0) /* ty=Tensor[(32), float32] */;
  %2 = divide(1f /* ty=float32 */, %1) /* ty=Tensor[(32), float32] */;
  %3 = multiply(%2, %graph_bn_gamma) /* ty=Tensor[(32), float32] */;
  %4 = nn.conv2d(%data, %graph_conv_weight, strides=[2, 2], padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 112, 112), float32] 

# Debug Pass

In [3]:
import numpy as np
import tvm
from tvm import te
import tvm.relay as relay

In [5]:
def example():
    shape = (1, 64, 54, 54)
    c_data = np.empty(shape).astype("float32")
    c = relay.const(c_data)
    
    weight = relay.var("weight", shape=(64, 64, 3, 3))
    x = relay.var("x", relay.TensorType((1, 64, 56, 56), "float32"))
    
    conv = relay.nn.conv2d(x, weight)
    y = relay.add(c, c)
    y = relay.multiply(y, relay.const(2, "float32"))
    y = relay.add(conv, y)
    z = relay.add(y, c)
    z1 = relay.add(y, c)
    z2 = relay.add(z, z1)
    return relay.Function([x, weight], z2)

In [6]:
# functions for optimization.
f = example()
mod = tvm.IRModule.from_expr(f)

print(type(mod))
#<class 'tvm.ir.module.IRModule'>

<class 'tvm.ir.module.IRModule'>


In [7]:
print(mod.show())

/Users/aiken/deep_learning/tvm_project/venv_tvm_project/lib/python3.10/site-packages/tvm-0.12.dev333+g7c06de52a-py3.10-macosx-12-arm64.egg/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/Users/aiken/deep_learning/tvm_project/venv_tvm_project/bin/python3.10 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


None


In [8]:
# (Debug) Pass
seq = tvm.transform.Sequential([relay.transform.FoldConstant(),
                                tvm.transform.PrintIR(),
                                relay.transform.EliminateCommonSubexpr(),
                                relay.transform.FuseOps(),
                                relay.transform.AlterOpLayout(),])



In [10]:
# 因為有使用AlterOpLayout()記憶體，所以需要註冊
# Memory Allocation
@relay.op.register_alter_op_layout("nn.conv2d", level=101)
def alter_conv2d(attrs, inputs, tinfos, out_type):
    data, weight = inputs
    new_attrs = dict(attrs)
    new_attrs["data_layout"] = "NCHW16c"
    return relay.nn.conv2d(data, weight, **new_attrs)

通過在 FoldConstant 之後插入 PrintIR，當 FoldConstant 完成時，pass infra 將轉儲模塊 IR。用戶可以在想要Debug任何 pass 之後插入它，以查看優化效果。

有一個更靈活的Debug機制。可以實現 PassInstrument 類來執行任意程式碼，不僅在每次傳遞之前和/或之後，還可以在進入/退出 PassContext 時執行。查看 Pass Instrument 瞭解更多信息。

這裡使用 tvm.instrument.pass_instrument() 裝飾器，實現 PassInsturment 類在每次 Pass 執行前打印 IR：




In [11]:
@tvm.instrument.pass_instrument
class PrintIR:
    """Print the name of the pass, the IR, only before passes execute."""

    def run_before_pass(self, mod, info):
        print("Running pass: {}", info)
        print(mod)


with tvm.transform.PassContext(opt_level=3, instruments=[PrintIR()]):
    with tvm.target.Target("llvm"):
        # Perform the optimizations.
        mod = seq(mod)

print(mod.show())

Running pass: {} The meta data of the pass - pass name: sequential, opt_level: 0, required passes: []

def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) {
  %0 = add(meta[relay.Constant][0], meta[relay.Constant][0]);
  %1 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]);
  %2 = multiply(%0, 2f);
  %3 = add(%1, %2);
  %4 = add(%3, meta[relay.Constant][0]);
  %5 = add(%3, meta[relay.Constant][0]);
  add(%4, %5)
}


Running pass: {} The meta data of the pass - pass name: FoldConstant, opt_level: 2, required passes: []

def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) {
  %0 = add(meta[relay.Constant][0], meta[relay.Constant][0]);
  %1 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]);
  %2 = multiply(%0, 2f);
  %3 = add(%1, %2);
  %4 = add(%3, meta[relay.Constant][0]);
  %5 = add(%3, meta[relay.Constant][0]);
  add(%4, %5)
}


Running pass: {} The meta data of the pass - pass name: InferType, opt_level: 0, require

[01:57:08] /Users/aiken/deep_learning/tvm/src/relay/ir/function.cc:141: PrintIR():
#[version = "0.0.5"]
def @main(%x: Tensor[(1, 64, 56, 56), float32] /* ty=Tensor[(1, 64, 56, 56), float32] */, %weight: Tensor[(64, 64, 3, 3), float32] /* ty=Tensor[(64, 64, 3, 3), float32] */) -> Tensor[(1, 64, 54, 54), float32] {
  %0 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %1 = add(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %2 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %3 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  add(%2, %3) /* ty=Tensor[(1, 64, 54, 54), float32] */
}

/* For debugging purposes the metadata section has been omitted.
 * If you would like to see the full metadata section you can set the 
 * option to `True` 

None
